In [10]:
import pandas as pd

from tkinter.messagebox import NO
from typing import Optional
import pandas as pd
import transformers
import time
from overlap_evaluate import (
    _print_score,
    get_scores,
    read_references,
    read_annotations,
    ANNOTATIONS,
)

from bert_score import score


/Users/calebkumar/repos/stanford/project/odqa-experiments/odqa-experiments/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
dpr_nq_test = pd.read_json("data/dpr/nq_test.json")

In [22]:
nq_test.head()

,question,answers
0,who got the first nobel prize in physics,['Wilhelm Conrad Röntgen']
1,when is the next deadpool movie being released,"['May 18 , 2018']"
2,which mode is used for short wave broadcast se...,"['Olivia', 'MFSK']"
3,the south west wind blows across nigeria between,['till September']
4,what does hp mean in war and order,['hit points or health points']


In [20]:
dataset = "nq"
annotations = read_annotations(f"data/{dataset}-annotations.jsonl")
references = read_references(f"data/{dataset}-test.qa.csv")
nq_test = pd.read_csv(
    f"data/{dataset}-test.qa.csv", sep="\t", names=["question", "answers"]
)

In [24]:
result_df = pd.merge(dpr_nq_test,nq_test, on='question')

In [36]:
result_df = result_df.drop("answers_x", axis=1)



,question,ctxs,answers_y
0,who got the first nobel prize in physics,"[{'id': 'wiki:718060', 'title': 'Wilhelm Rönt...",['Wilhelm Conrad Röntgen']
1,when is the next deadpool movie being released,"[{'id': 'wiki:18286085', 'title': 'Deadpool (f...","['May 18 , 2018']"
2,which mode is used for short wave broadcast se...,"[{'id': 'wiki:685104', 'title': 'Shortwave rad...","['Olivia', 'MFSK']"
3,the south west wind blows across nigeria between,"[{'id': 'wiki:283802', 'title': 'Geography of ...",['till September']
4,what does hp mean in war and order,"[{'id': 'wiki:13754811', 'title': 'Generals Or...",['hit points or health points']
...,...,...,...
3605,who challenged the aristotelian model of a geo...,"[{'id': 'wiki:1828973', 'title': 'Heliocentris...",['Copernicus']
3606,when was the miraculous journey of edward tula...,"[{'id': 'wiki:9638146', 'title': 'The Miraculo...","['2006', 'March 30 , 2006']"
3607,character in macbeth who is murdered and appea...,"[{'id': 'wiki:4403360', 'title': 'Lady Macbeth...","['Lord Banquo', 'Banquo']"
3608,when was as you like it first performed,"[{'id': 'wiki:13533242', 'title': 'As You Like...",['1603']


In [37]:
result_df = result_df.rename(columns={"answers_y": "answers"})

,question,answers_x,ctxs,answers
0,who got the first nobel prize in physics,[Wilhelm Conrad Röntgen],"[{'id': 'wiki:718060', 'title': 'Wilhelm Rönt...",['Wilhelm Conrad Röntgen']
1,when is the next deadpool movie being released,"[May 18 , 2018]","[{'id': 'wiki:18286085', 'title': 'Deadpool (f...","['May 18 , 2018']"
2,which mode is used for short wave broadcast se...,"[Olivia, MFSK]","[{'id': 'wiki:685104', 'title': 'Shortwave rad...","['Olivia', 'MFSK']"
3,the south west wind blows across nigeria between,[till September],"[{'id': 'wiki:283802', 'title': 'Geography of ...",['till September']
4,what does hp mean in war and order,[hit points or health points],"[{'id': 'wiki:13754811', 'title': 'Generals Or...",['hit points or health points']
...,...,...,...,...
3605,who challenged the aristotelian model of a geo...,[Copernicus],"[{'id': 'wiki:1828973', 'title': 'Heliocentris...",['Copernicus']
3606,when was the miraculous journey of edward tula...,"[2006, March 30 , 2006]","[{'id': 'wiki:9638146', 'title': 'The Miraculo...","['2006', 'March 30 , 2006']"
3607,character in macbeth who is murdered and appea...,"[Lord Banquo, Banquo]","[{'id': 'wiki:4403360', 'title': 'Lady Macbeth...","['Lord Banquo', 'Banquo']"
3608,when was as you like it first performed,[1603],"[{'id': 'wiki:13533242', 'title': 'As You Like...",['1603']


In [38]:
result_df = result_df.drop("answers_x", axis=1)


In [55]:
test_df = pd.read_csv(
            f"data/{dataset}-test-ctxs.qa.csv"
        )

In [56]:
test_df.head()

,question,ctxs,answers
0,who got the first nobel prize in physics,"[{'id': 'wiki:718060', 'title': 'Wilhelm Rönt...",['Wilhelm Conrad Röntgen']
1,when is the next deadpool movie being released,"[{'id': 'wiki:18286085', 'title': 'Deadpool (f...","['May 18 , 2018']"
2,which mode is used for short wave broadcast se...,"[{'id': 'wiki:685104', 'title': 'Shortwave rad...","['Olivia', 'MFSK']"
3,the south west wind blows across nigeria between,"[{'id': 'wiki:283802', 'title': 'Geography of ...",['till September']
4,what does hp mean in war and order,"[{'id': 'wiki:13754811', 'title': 'Generals Or...",['hit points or health points']


In [57]:
test_df.iloc[0]

question             who got the first nobel prize in physics
ctxs        [{'id': 'wiki:718060', 'title': 'Wilhelm Rönt...
answers                            ['Wilhelm Conrad Röntgen']
Name: 0, dtype: object